In [3]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00


In [4]:
pip install google-generativeai

In [6]:
pip install chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 7.7 MB/s eta 

In [7]:
pip install "chromadb[recommended]" # 推薦安裝完整版本，包含更多依賴

In [28]:
import PyPDF2
import re
import chromadb
import google.generativeai as genai
from chromadb.utils import embedding_functions
import os

# Ensure your API Key is set as an environment variable GOOGLE_API_KEY
# If not, add os.environ["GOOGLE_API_KEY"] = "Your API Key" at the top of the code.
if "GOOGLE_API_KEY" not in os.environ:
    print("Warning: GOOGLE_API_KEY environment variable not set. Please set it or hardcode it (not recommended).")
    # For demonstration, you might temporarily hardcode it here if you understand the security implications.
    # os.environ["GOOGLE_API_KEY"] = "YOUR_ACTUAL_GEMINI_API_KEY_HERE"

genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

gemini_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(
    api_key=os.environ.get("GOOGLE_API_KEY"),
    model_name="models/embedding-001"
)

# Initialize Gemini GenerativeModel
# 'gemini-pro' is a suitable model for text generation
try:
    gemini_model = genai.GenerativeModel('models/gemini-1.5-flash-latest')
except Exception as e:
    print(f"Error initializing Gemini GenerativeModel: {e}")
    gemini_model = None # Set to None if initialization fails


def extract_text_from_pdf(pdf_path):
    """
    Extracts all text from the specified PDF file.
    Args:
        pdf_path (str): The path to the PDF file.
    Returns:
        str: All extracted text from the PDF, or None if an error occurs.
    """
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                extracted_page_text = page.extract_text()
                if extracted_page_text:
                    text += extracted_page_text + "\n\n"
            return text
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return None

def split_text_into_chunks_fixed_length(text, chunk_size=300, chunk_overlap=50):
    """
    Splits text into fixed-length chunks with overlap.
    Args:
        text (str): The complete text string.
        chunk_size (int): The maximum number of characters per text chunk.
        chunk_overlap (int): The number of characters to overlap between adjacent chunks.
    Returns:
        list: A list of strings, where each string is a text chunk.
    """
    if not text:
        return []

    cleaned_text = text.replace('\n', ' ').replace('\r', ' ').strip()
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text) # Replace multiple spaces with a single space

    chunks = []
    start_index = 0
    while start_index < len(cleaned_text):
        end_index = start_index + chunk_size
        if end_index > len(cleaned_text):
            chunks.append(cleaned_text[start_index:])
            break

        # Try to split at the end of a sentence to avoid truncating sentences
        split_point = cleaned_text.rfind('.', start_index, end_index)
        if split_point == -1:
            split_point = cleaned_text.rfind('。', start_index, end_index)
        if split_point == -1:
             split_point = cleaned_text.rfind(' ', start_index, end_index)

        if split_point > start_index + chunk_overlap:
            current_chunk = cleaned_text[start_index:split_point + 1].strip()
            chunks.append(current_chunk)
            start_index = split_point + 1 - chunk_overlap
        else:
            current_chunk = cleaned_text[start_index:end_index].strip()
            chunks.append(current_chunk)
            start_index += chunk_size - chunk_overlap

        start_index = max(0, start_index)

    return [chunk for chunk in chunks if chunk]


def create_chroma_db(documents, db_path="./chroma_db", collection_name="pdf_qa_collection"):
    """
    Creates and populates a ChromaDB database.
    Args:
        documents (list): A list containing text chunks to be stored.
        db_path (str): The path where the ChromaDB database will be stored.
        collection_name (str): The name of the database collection.
    Returns:
        chromadb.api.models.Collection.Collection: The created or retrieved ChromaDB collection.
    """
    try:
        client = chromadb.PersistentClient(path=db_path)
        collection = client.get_or_create_collection(
            name=collection_name,
            embedding_function=gemini_ef
        )

        if collection.count() == 0:
            ids = [f"doc_{i}" for i in range(len(documents))]
            collection.add(
                documents=documents,
                ids=ids
            )
            print(f"Successfully added {len(documents)} documents to ChromaDB collection '{collection_name}'.")
        else:
            print(f"ChromaDB collection '{collection_name}' already contains {collection.count()} documents, skipping addition.")

        print(f"Database storage path: {db_path}")
        return collection

    except Exception as e:
        print(f"Error creating or populating ChromaDB: {e}")
        return None

def get_relevant_passage(query, db_collection, n_results=3):
    """
    Retrieves the most relevant passages from the ChromaDB collection based on the query.
    Args:
        query (str): The user's query string.
        db_collection (chromadb.api.models.Collection.Collection): The ChromaDB collection object.
        n_results (int): The number of relevant passages to retrieve.
    Returns:
        list: A list of strings containing the most relevant text passages.
    """
    try:
        results = db_collection.query(
            query_texts=[query],
            n_results=n_results,
            include=['documents']
        )
        relevant_documents = results.get('documents', [[]])[0]
        return relevant_documents
    except Exception as e:
        print(f"Error retrieving relevant passages: {e}")
        return []

# --- Added build_rag_prompt function ---
def build_rag_prompt(query, relevant_passages):
    """
    Constructs the RAG prompt for the generative model.
    Args:
        query (str): The user's original query.
        relevant_passages (list): A list of relevant text passages retrieved from the vector database.
    Returns:
        str: The combined prompt string.
    """
    prompt = f"You are a question answering system. Please answer the question based on the provided context. If there is not enough information in the context, please state that you cannot answer.\n\n"
    prompt += "Context:\n"
    for i, passage in enumerate(relevant_passages):
        prompt += f"Passage {i+1}: {passage}\n"
    prompt += f"\nQuestion: {query}\n"
    prompt += "Answer:"
    return prompt

# --- Integrate all steps and execute ---

if __name__ == "__main__":
    pdf_file_path = "AI Team2 7.4報告.pdf"
    db_path = "./my_pdf_chroma_db"
    collection_name = "ai_team2_report_qa"

    # 1. Extract text from PDF
    full_pdf_text = extract_text_from_pdf(pdf_file_path)

    if full_pdf_text:
        print("Successfully extracted text from PDF.")

        # 2. Split text into chunks
        chunks = split_text_into_chunks_fixed_length(full_pdf_text, chunk_size=300, chunk_overlap=50)
        print(f"Split into {len(chunks)} text chunks.")

        # 3. Create and populate ChromaDB
        chroma_collection = create_chroma_db(documents=chunks,
                                             db_path=db_path,
                                             collection_name=collection_name)

        if chroma_collection and gemini_model:
            print("ChromaDB collection and Gemini model are ready.")

            # 4. Perform retrieval
            user_query = "What are the key features of the project?"
            # For testing, we still use this query, but note that the previously retrieved passages were not highly relevant
            # It is recommended to try other queries, such as "What is Chinese medicine?" or "What technologies are used?"

            print(f"\n--- Retrieving relevant passages, query: \"{user_query}\" ---")
            # Adjust n_results to get more context, which may improve answer accuracy
            retrieved_passages = get_relevant_passage(user_query, chroma_collection, n_results=3)

            if retrieved_passages:
                print("Retrieved relevant passages:")
                for i, passage in enumerate(retrieved_passages):
                    print(f"Passage {i+1}:\n{passage}\n---")

                # 5. Build RAG prompt
                rag_prompt = build_rag_prompt(user_query, retrieved_passages)
                print("\n--- Constructed RAG prompt ---")
                print(rag_prompt)

                # 6. Generate answer using Gemini model
                print("\n--- Generating answer using Gemini model ---")
                try:
                    response = gemini_model.generate_content(rag_prompt)
                    # Check if response has text attribute to avoid errors
                    if response.candidates and response.candidates[0].content.parts:
                        print(response.candidates[0].content.parts[0].text)
                    else:
                        print("Gemini model failed to generate a valid answer.")
                        print(f"Original response: {response}") # Print original response for debugging
                except Exception as e:
                    print(f"Error generating answer from Gemini model: {e}")
            else:
                print("No relevant passages found, cannot construct prompt and generate answer.")
        else:
            print("Failed to create ChromaDB collection or initialize Gemini model.")
    else:
        print("Failed to extract text from PDF.")

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Successfully extracted text from PDF.
Split into 9 text chunks.
ChromaDB collection 'ai_team2_report_qa' already contains 9 documents, skipping addition.
Database storage path: ./my_pdf_chroma_db
ChromaDB collection and Gemini model are ready.

--- Retrieving relevant passages, query: "What are the key features of the project?" ---
Retrieved relevant passages:
Passage 1:
ublic ●Professionals may require higher precision Future Work ●Build a curated database ●Improve classiﬁcation for better accuracy Thank you for your attention! Q&A
---
Passage 2:
3 Connect Gemini AI to achieve image recognition . next(iter(uploaded)) How We Built It ? Program demonstration (colab) ○ Evaluation Evaluation Metrics ：Accuracy Results ：Our model achieved 80% accuracy .
---
Passage 3:
rt capturing images Technologies ○ STEP1 Use ChatGPT to build the program framework for the AI identiﬁcation system. STEP2 Develop the camera interface functionality .STEP4 Perform program debugging and testing .STEP3 Connect 